# Instructions

1. git clone the repo and upload the whole repo to google drive
2. use Google Colab to execute the code (including model training, inference, and applying VAD)

# Section 1: DLRM training && inference

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# enter your working directory

# %cd /content/gdrive/MyDrive/your_folder/VAD/dlrm/

In [ ]:
# install the required package

# !pip install "git+https://github.com/mlperf/logging.git@beaf26d"

In [ ]:
# command to train DLRM model && do inference on test / validation dataset
# to fully reproduce the result, please train the model on 15M data instead of 100k example data


# %%shell

# for VAR in {1..1}
# do 
#     python dlrm_s_pytorch.py \
#         --arch-sparse-feature-size=16 \
#         --arch-mlp-bot="13-512-256-64-16" \
#         --arch-mlp-top="512-256-1" \
#         --data-generation=dataset \
#         --data-set=kaggle \
#         --raw-data-file=./input/train_last_100k_as_example.txt \
#         --processed-data-file=./input/kaggleAdDisplayChallenge_processed.npz \
#         --save-model=./model/model_example.pt \
#         --numpy-rand-seed=${VAR} \
#         --data-randomize=total \
#         --lr-num-warmup-steps=0 \
#         --lr-decay-start-step=0 \
#         --loss-function=bce \
#         --round-targets=True \
#         --mlperf-logging \
#         --nepochs=2 \
#         --learning-rate=0.1 \
#         --mini-batch-size=128 \
#         --print-freq=81920 \
#         --print-time \
#         --test-mini-batch-size=16384 \
#         --test-num-workers=4 \
#         --test-freq=655360

#     python dlrm_s_pytorch.py \
#         --arch-sparse-feature-size=16 \
#         --arch-mlp-bot="13-512-256-64-16" \
#         --arch-mlp-top="512-256-1" \
#         --data-generation=dataset \
#         --data-set=kaggle \
#         --raw-data-file=./input/train_last_100k_as_example.txt \
#         --processed-data-file=./input/kaggleAdDisplayChallenge_processed.npz \
#         --load-model=./model/model_example.pt \
#         --dump-json-file=./result/result_example.json \
#         --test-data-split=test \
#         --inference-only \
#         --data-randomize=total \
#         --lr-num-warmup-steps=0 \
#         --lr-decay-start-step=0 \
#         --loss-function=bce \
#         --round-targets=True \
#         --mlperf-logging \
#         --nepochs=1 \
#         --learning-rate=0.1 \
#         --mini-batch-size=128 \
#         --print-freq=81920 \
#         --print-time \
#         --test-mini-batch-size=16384 \
#         --test-num-workers=4 \
#         --test-freq=655360

#     python dlrm_s_pytorch.py \
#         --arch-sparse-feature-size=16 \
#         --arch-mlp-bot="13-512-256-64-16" \
#         --arch-mlp-top="512-256-1" \
#         --data-generation=dataset \
#         --data-set=kaggle \
#         --raw-data-file=./input/train_last_100k_as_example.txt \
#         --processed-data-file=./input/kaggleAdDisplayChallenge_processed.npz \
#         --load-model=./model/model_example.pt \
#         --dump-json-file=./result/result_example_val.json \
#         --test-data-split=val \
#         --inference-only \
#         --data-randomize=total \
#         --lr-num-warmup-steps=0 \
#         --lr-decay-start-step=0 \
#         --loss-function=bce \
#         --round-targets=True \
#         --mlperf-logging \
#         --nepochs=1 \
#         --learning-rate=0.1 \
#         --mini-batch-size=128 \
#         --print-freq=81920 \
#         --print-time \
#         --test-mini-batch-size=16384 \
#         --test-num-workers=4 \
#         --test-freq=655360
# done

# Section 2: Apply VAD

In [ ]:
# %cd /content/gdrive/MyDrive/your_folder/VAD/

In [ ]:
import codecs, json
import numpy as np
import tensorflow as tf

def get_y_and_p(file_name):
    obj_text = codecs.open(file_name, 'r', encoding='utf-8').read()
    data = json.loads(obj_text)
    y = np.squeeze(np.array(data['targets']))
    p = np.squeeze(np.array(data['scores']))
    return y, p

def assert_test_val_different(y_test, y_val):
    # make sure test data and val data are indeed different (i.e. no bug when do model inference)
    if len(y_test) == len(y_val):
        assert np.sum(y_test == y_val) < len(y_val)

def read_data_from_json(
    json_name, 
    json_name_val, 
    num_group,
    alphas_zero,
):

    def get_test_and_val(y_1, p_1, y_2, p_2):
        assert_test_val_different(y_1, y_2)
        y = np.hstack((y_1, y_2))
        p = np.hstack((p_1, p_2))
        num_examples = y.shape[0]
        num_examples_val = num_examples // 10
        num_examples -= num_examples_val
        return y[:num_examples], p[:num_examples], y[num_examples:], p[num_examples:]

    y_1, p_1 = get_y_and_p(json_name.format(1))
    y_2, p_2 = get_y_and_p(json_name_val.format(1))
    y, p, y_val, p_val = get_test_and_val(y_1, p_1, y_2, p_2)
    num_examples = y.shape[0]
    num_examples_val = y_val.shape[0]

    p_predicted_subgroup = np.zeros((num_examples, num_group))
    y_predicted_subgroup = np.zeros((num_examples, num_group))
    after_selection_calibration_val = {}
    for alpha_zero in alphas_zero:
        alpha_key = int(1000 * alpha_zero)
        after_selection_calibration_val[alpha_key] = np.zeros(num_group)
    
    for i in range(num_group):
        y_1, p_1 = get_y_and_p(json_name.format(i+1))
        y_2, p_2 = get_y_and_p(json_name_val.format(i+1))
        y, p, y_val, p_val = get_test_and_val(y_1, p_1, y_2, p_2)
        y_predicted_subgroup[:, i] = y
        p_predicted_subgroup[:, i] = p
        for alpha_zero in alphas_zero:
            ind_val = np.argpartition(p_val, -int(num_examples_val * alpha_zero))[-int(num_examples_val * alpha_zero):]
            after_selection_calibration_val[int(1000 * alpha_zero)][i] = np.sum(p_val[ind_val]) / np.sum(y_val[ind_val])

    # make sure data is correct
    for i in range(num_group):
        assert np.sum(y_predicted_subgroup[:,0] == y_predicted_subgroup[:,i]) == num_examples

    return p_predicted_subgroup, y_predicted_subgroup[:, 0], after_selection_calibration_val

def get_ood_construct_model_predictions():
    json_name_nn_select = './dlrm/result/result_model_select.json'
    json_name_nn_select_val = './dlrm/result/result_model_select_val.json'
    y_1, p_1 = get_y_and_p(json_name_nn_select)
    y_2, p_2 = get_y_and_p(json_name_nn_select_val)
    assert_test_val_different(y_1, y_2)
    y = np.hstack((y_1, y_2))
    p = np.hstack((p_1, p_2))
    return y, p

json_name = "./dlrm/result/result_model_{}.json"
json_name_val = "./dlrm/result/result_model_{}_val.json"

# The paper replicates the experiment 40 times, so in order to reproduce the results,
# you should train 40 * 2 = 80 models and set num_group = 80
num_group = 4
alphas_zero = [0.02, 0.05, 0.1]

bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)
ood_y, ood_p = get_ood_construct_model_predictions()

p_predicted_subgroup_original, y_original, after_selection_calibration_val = read_data_from_json(
    json_name, 
    json_name_val, 
    num_group,
    alphas_zero,
)

In [ ]:
def preprocess_test_data(
    p_predicted_subgroup, 
    y,
    do_p_selection,
    ood_p,
    ood_y,
):
    num_examples = p_predicted_subgroup.shape[0]
    assert np.sum(ood_y[:num_examples] == y) == len(y)
    selected_ind = list(range(num_examples))
    if do_p_selection:
        selected_ind = []
        for i in range(num_examples):
            select = np.random.binomial(1, 1 - ood_p[i])
            if select > 0.5:
                selected_ind.append(i)
    selected_ind = np.array(selected_ind)
    print(f"Selected {len(selected_ind)} samples after test pre-process ({int(100 * len(selected_ind) // num_examples)}% data)")
    print("After pre-process test positive ratio: ", np.sum(y[selected_ind]) / len(selected_ind))
    print()
    return p_predicted_subgroup[selected_ind], y[selected_ind]

do_p_selection = False
num_group_bootstrap = 2
np.random.seed(1)

result_keys = [
    'total_calibration',
    'positive_ratio_after_selection',
    'Vanilla',
    'Validation',
    'Validation_50',
    'Validation_100',
    'DBLL',
    'VAD',
    'VAD prob',
    'Ensemble',
    'Lambda logit',
    'Lambda prob',
    'Mu logit',
    'Mu prob',
    'Log Loss Logit Improvement',
    'Log Loss Prob Improvement',
]

p_predicted_subgroup, y = preprocess_test_data(
    p_predicted_subgroup_original, 
    y_original,
    do_p_selection,
    ood_p,
    ood_y,
)

In [ ]:
from VAD_util import calculate_lambda, prediction_transformation

def generate_report(
    y, 
    p_predicted_subgroup, 
    after_selection_calibration_val, 
    alpha, 
    print_individual_result,
    num_group_bootstrap,
):
    def append_result(result, curr):
        for result_key in result_keys:
            result[result_key].append(curr[result_key])
        return result

    num_examples, num_group = p_predicted_subgroup.shape
    num_examples_test_val = num_examples // 10
    num_examples -= num_examples_test_val
    p_predicted_subgroup_test_val = p_predicted_subgroup[:num_examples_test_val, :]
    p_predicted_subgroup = p_predicted_subgroup[num_examples_test_val:, :]
    y = y[num_examples_test_val:]
    assert num_examples == p_predicted_subgroup.shape[0]

    result = {}
    for result_key in result_keys:
        result[result_key] = []

    p_avg = np.mean(p_predicted_subgroup, axis=1)

    for i in range(num_group):
        if i >= num_group // num_group_bootstrap:
            continue
        p = p_predicted_subgroup[:, i]
        p_next = p_predicted_subgroup[:, (i + num_group // num_group_bootstrap) % num_group] 
        ind = np.argpartition(p, -int(num_examples * alpha))[-int(num_examples * alpha):]
        p_ground_truth = p_avg

        pos_rate = np.sum(y) / num_examples
        NE_denominator = - (pos_rate * np.log(pos_rate) + (1 - pos_rate) * np.log(1 - pos_rate))

        p_predicted_subgroup_test_val_choosen = np.zeros((num_examples_test_val, num_group_bootstrap))
        for j in range(num_group_bootstrap):
            p_predicted_subgroup_test_val_choosen[:, j] = p_predicted_subgroup_test_val[:, (i + j * num_group // num_group_bootstrap) % num_group]
        lambda_p_logit = calculate_lambda(p_predicted_subgroup_test_val_choosen, p_predicted_subgroup_test_val[:, i], 'logit')
        lambda_p_prob = calculate_lambda(p_predicted_subgroup_test_val_choosen, p_predicted_subgroup_test_val[:, i], 'probability')
        p_mean_test_val = np.mean(p_predicted_subgroup_test_val[:, i])
        p_mean_logit_test_val = np.mean(np.log(p_predicted_subgroup_test_val[:, i]/(1-p_predicted_subgroup_test_val[:, i])))
        refined_prediction_logit = prediction_transformation(p, ind, lambda_p_logit, 'logit', p_mean_test_val, p_mean_logit_test_val)
        refined_prediction_prob = prediction_transformation(p, ind, lambda_p_prob, 'probability', p_mean_test_val, p_mean_logit_test_val)
        log_loss_original = bce(y[ind], p[ind]).numpy()
        log_loss_logit = bce(y[ind], refined_prediction_logit).numpy()
        log_loss_prob = bce(y[ind], refined_prediction_prob).numpy()
        log_loss_logit_improve = (log_loss_logit - log_loss_original) / log_loss_original * 100
        log_loss_prob_improve = (log_loss_prob - log_loss_original) / log_loss_original * 100

        curr = {
            'total_calibration': np.sum(p) / np.sum(y),
            'positive_ratio_after_selection': np.sum(y[ind]) / len(y[ind]),
            'Vanilla': np.sum(p[ind]) / np.sum(y[ind]),
            'Validation': np.sum(p[ind]) / np.sum(y[ind]) / after_selection_calibration_val[20][i],
            'Validation_50': np.sum(p[ind]) / np.sum(y[ind]) / after_selection_calibration_val[50][i],
            'Validation_100': np.sum(p[ind]) / np.sum(y[ind]) / after_selection_calibration_val[100][i],
            'DBLL': np.sum(p_next[ind]) / np.sum(y[ind]),
            'VAD': np.sum(refined_prediction_logit) / np.sum(y[ind]),
            'VAD prob': np.sum(refined_prediction_prob) / np.sum(y[ind]),
            'Ensemble': np.sum(p_ground_truth[ind]) / np.sum(y[ind]),
            'Lambda logit': lambda_p_logit,
            'Lambda prob': lambda_p_prob,
            'Mu logit': p_mean_logit_test_val,
            'Mu prob': p_mean_test_val,
            'Log Loss Logit Improvement': log_loss_logit_improve,
            'Log Loss Prob Improvement': log_loss_prob_improve,
        }
        result = append_result(result, curr)
        
        log_loss = bce(y, p).numpy()

        if print_individual_result:
            print("LogLoss: ", log_loss)
            print("NE: ", log_loss / NE_denominator)
            for result_key in result_keys:
                print(result_key, ": ", curr[result_key])
            print()

    print("Num Group: ", num_group)
    print("Alpha: ", alpha)
    for result_key in result_keys:
        result[result_key] = np.array(result[result_key])
        print("mean of ", result_key, ": ", np.mean(result[result_key]))
    print()

    return result

def generate_multiple_report(p_predicted_subgroup, y, after_selection_calibration_val):
    num_group = p_predicted_subgroup.shape[1]
    report = {}
    alphas = [0.02, 0.05, 0.1]
    for alpha in alphas:
        result = generate_report(
            y, 
            p_predicted_subgroup, 
            after_selection_calibration_val, 
            alpha, 
            False,
            num_group_bootstrap,
        )
        alpha_key = int(1000 * alpha)
        report[alpha_key] = {}
        for result_key in reported_result_keys:
            curr_result = result[result_key]
            total_num_result = curr_result.shape[0]
            report[alpha_key][result_key] = (np.mean(curr_result), np.std(curr_result) / np.sqrt(total_num_result))
    return report


reported_result_keys = [
    'Vanilla',
    'Validation',
    'DBLL',
    'VAD',
    'Ensemble',
]

final_report = generate_multiple_report(p_predicted_subgroup, y, after_selection_calibration_val)
